# Underwriting Guidelines Project

The goal of this Jupyter Notebook is to develop a logical framework capable of determining whether an arbitrary underwriter has the authority to quote an insurance submission. This problem is approached using principles from object-oriented programming, data structures, and algorithm design with an emphasis on time and space complexity for guideline lookups and authority verification. 

### Objective

The task is to design an `is_authorized()` method. Given an arbitrary underwriter and submission, it returns whether the submission falls within the underwriter’s decision authority, and if not, identifies the reason. The method is used as underwriter.is_authorized(submission). Later sections simulate randomized submissions to test the behavior of both underwriters and underwriting managers.

#### Class Design

To achieve this, two main classes are defined:

1. Underwriter: representing the underwriter’s authority parameters. Each underwriter has:
- A name
- A maximum premium authority limit. This is typically based on modified premium.
- Minimum and maximum experience rating modification (XMOD) bounds
- A list of territories they are permitted to write business in
- A managerial level indicator, since certain class codes require managerial authority to approve. If their managerial level is 1 then they are a manager.

Method: `is_authorized()`: This method evaluates a submission against these rules, checking:
- If the submission’s modified premium is within authority
- If the experience rating (XMOD) is within allowable limits
- If all submission territories are authorized
- If the submission contains class codes requiring managerial referral

The method will return false at the first failure, checking first modified premium, Xmod, class code, and lastly territory. This order was selected because the complexity of the subsequent verficiation is larger. For instance, the premium verification is a simple check, `submission.modified_premium > underwriter.premium_authority` whereas the class code authority performs a lookup into the underwriting guidelines dictionary, and a lookup into the incidental dictionary, and performs this lookup for each class code in the submission. For scalability, reducing the algorithmic complexity is important. If there were thousands of submissions, we would be performing many unnecessary checks. As a sidenote, this is referred to as the partial authorization function. One can also define a 'full' authorization function that returns every single failure. 

2. Submission: representing an account or risk being quoted. A submission includes:
- A name
- A dictionary of class codes with a corresponding exposure and territory mix.
- An Xmod

The submission class also computes:
- Manual Premium: The sum of `exposure * class_rate`
- Modified Premium: `manual_premium * Xmod`
- The share of total manual premium attributable to each class code
- The shore of total manual premium attributable to each territory

#### Data Structure Design

##### Guidelines
A global underwriting guidelines dictionaries was created to define the underwriting rules per class code. Each class code is designated as either standard or managerial, with an optional additional incidental threshold percentage. This allows flexibility for classes that require referral only when their exposure exceeds a defined threshold. The dictionary was composed of the form `{'class_code': {'territory': territory, 'rate': rate, 'managerial': managerial 0/1, 'incidental': incidental_percentage}}`. 

The exposure basis in the submissions class is of the form `{class_code: {'territory': territory, 'exposure': exposure}}`. E.g. `{8810: {'territory': 'CA', 'exposure': 1250000}}`. This may seem awkward but a dictionary data structure allows for average-case $O(1)$ lookups. A dictionary structure allows for readability as well as avoiding issues with indexing if we were to use a list. As an example, `submission.exposure_basis['8810']['territory']` is interpretable as the territory for class code 8810 without knowlege of positional indexing. 

#### is_authorized() algorithmic analysis

The `is_authorized()` method performs the following operations:
1. Checks if the submission modified premium exceeds the underwriter modified premium authority.
2. Checks if the submission Xmod is within the underwritier Xmod authority range
3. Checks if the territory distribution is within the underwriters authority based on AOS premium percentage.
4. Checks if the underwriter is a manager. If the underwriter is a manager, then the underwriter has authority for all managerial class codes. If the underwriter is not a manager, proceed to step 5.
5. For every class code, checks if the class code is a managerial class code. If the class code is a standard class code, the account is in the underwriters authority. If not, proceed to step 6
6. Checks if the managerial class code is incidental as defined in the underwriting guidelines. If it is non incidental and exceeds the incidental guideline, it is outside of the underwriting authority. If it is incidental, it is in the underwriters authority.

The reason that is_authorized() method is performed in these steps is because the steps iteratively get more computationally complex. Steps (1) and (2) are simple whereas steps 3-6 involve lookups into the underwriting guidelines table and the calculation of different fractions. This progression ensures efficient termination on simple failures and postpones heavier computations until necessary.

As a note, this framework does not account for other risks that could appear in a submission such as employee concentration, group transportation, perils, terrorism risk, or catastrophe. In addition the qualitative assessment of a risk might merit a referral for other reasons. This framework is helpful for helping underwriters decide if they have decision authority but is not a ground truth authority model. 

In the next lines of code, the WCIRB pure premium rates are imported as a base rate dictionary and the managerial/incidental guidelines dictionaries are populated. The NCCI does not have public class codes, so the WCIRB class codes are recoded and base rates are changed by $\pm 5\%$. These guidelines are randomized, with a 10% chance of each class code being a managerial class code, and the incidental thresholds being set according to $ U \sim \text{unif}(0.05,0.25)$ rounded to 4 decimal places. A brief snapshot of the guidelines will also be displayed.

In [1]:
import csv
import random
import pandas as pd

WCIRB_pure_premium_rates = {} #Start with an empty dictionary, we will populate this with the WCIRB pure premium rates as class code - pure premium rate pairs.
with open("approved_09012025_pure_premium_rates.csv", newline = "") as pure_premiums:
    reader = csv.reader(pure_premiums)
    for row in reader:
        class_code, rate = row
        WCIRB_pure_premium_rates[class_code] = float(rate)

WCIRB_managerial_guideline_dictionary = {}
WCIRB_incidental_threshold_guidelines_dictionary = {}

random.seed(42)
for class_code in WCIRB_pure_premium_rates.keys():
    # 10% chance of being a managerial class code
    is_managerial = random.random() < 0.10
    WCIRB_managerial_guideline_dictionary[class_code] = 1 if is_managerial else 0
    if is_managerial:
        threshold = round(random.uniform(0.05, 0.25), 4)  # fraction, not %
        WCIRB_incidental_threshold_guidelines_dictionary[class_code] = threshold

NCCI_pure_premium_rates = {}
for class_code, pure_premium_rate in WCIRB_pure_premium_rates.items():
    NCCI_pure_premium_rates.update({class_code + str(random.choice([0, 1])): round(float(pure_premium_rate * random.uniform(0.95, 1.05)),2)})

NCCI_managerial_guideline_dictionary = {}
NCCI_incidental_threshold_guidelines_dictionary = {}

for class_code in NCCI_pure_premium_rates.keys():
    # 10% chance of being a managerial class code
    is_managerial = random.random() < 0.10
    NCCI_managerial_guideline_dictionary[class_code] = 1 if is_managerial else 0
    if is_managerial:
        threshold = round(random.uniform(0.05, 0.25), 4)  # fraction, not %
        NCCI_incidental_threshold_guidelines_dictionary[class_code] = threshold

# consolidate into a single data structure, dictionary of dictionaries structure; 
# {class_code: {"territory": "CA"/"AOS", "rate": float, "managerial": 0/1, "incidental": float}}
class_guidelines = {}

# WCIRB (CA)
for cc, rate in WCIRB_pure_premium_rates.items():
    managerial = WCIRB_managerial_guideline_dictionary.get(cc, 0)
    incidental = WCIRB_incidental_threshold_guidelines_dictionary.get(cc, 0.0) if managerial == 1 else 0.0
    class_guidelines[cc] = {
        "territory": "CA",
        "rate": float(rate),
        "managerial": managerial,
        "incidental": float(incidental)
    }

# NCCI (AOS)
for cc, rate in NCCI_pure_premium_rates.items():
    managerial = NCCI_managerial_guideline_dictionary.get(cc, 0)
    incidental = NCCI_incidental_threshold_guidelines_dictionary.get(cc, 0.0) if managerial == 1 else 0.0
    class_guidelines[cc] = {
        "territory": "AOS",
        "rate": float(rate),
        "managerial": managerial,
        "incidental": float(incidental)
    }

print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [2]:
print("Underwriting Guideline Sample:")
for k, v in list(class_guidelines.items())[500:510]:
   print(k, v)

Underwriting Guideline Sample:
00401 {'territory': 'AOS', 'rate': 3.81, 'managerial': 0, 'incidental': 0.0}
00410 {'territory': 'AOS', 'rate': 4.29, 'managerial': 0, 'incidental': 0.0}
00421 {'territory': 'AOS', 'rate': 5.04, 'managerial': 0, 'incidental': 0.0}
00451 {'territory': 'AOS', 'rate': 4.58, 'managerial': 0, 'incidental': 0.0}
00500 {'territory': 'AOS', 'rate': 6.09, 'managerial': 0, 'incidental': 0.0}
00791 {'territory': 'AOS', 'rate': 2.54, 'managerial': 0, 'incidental': 0.0}
00961 {'territory': 'AOS', 'rate': 4.61, 'managerial': 0, 'incidental': 0.0}
01061 {'territory': 'AOS', 'rate': 10.88, 'managerial': 0, 'incidental': 0.0}
01711 {'territory': 'AOS', 'rate': 6.07, 'managerial': 1, 'incidental': 0.1864}
01721 {'territory': 'AOS', 'rate': 3.73, 'managerial': 0, 'incidental': 0.0}


In the following lines of code, we define the underwriter and submission class, as well as the methods such as the `is_authorized()`.

In [3]:
class Underwriter():
    def __init__(self, name, premium_authority, xmod_min_authority, xmod_max_authority, territory_authority, managerial_level):
        self.name = name
        self.premium_authority = premium_authority
        self.xmod_min_authority = xmod_min_authority
        self.xmod_max_authority = xmod_max_authority
        self.territory_authority = territory_authority  #CA/AOS authority as a scalar number. This is the AOS limit per underwriter. E.g. 0.75 = 75% AOS authority. 
        self.managerial_level = managerial_level
# -------------------------------------------------------------------------
    def is_authorized(self, submission):
        # Check premium
        if submission.modified_premium > self.premium_authority:
            return False, "Referral: Premium"
        # Check Xmod
        if not (self.xmod_min_authority <= submission.xmod <= self.xmod_max_authority):
            return False, "Referral: Xmod"
            
        # 3) Territory cap (AOS share)
        dist = submission.calc_territory_distribution()  # e.g., {"CA": 0.72, "AOS": 0.28}
        aos_share = float(dist.get("AOS", 0.0))
        cap = float(getattr(self, "territory_authority", 0.25))
        if aos_share > cap:
            return False, f"Referral: Territory (AOS {aos_share:.1%} > {cap:.0%})"
        # 4) Class code incidental limits (skip if manager)
        if getattr(self, "managerial_level", 0) != 1:
            violating = []
            for cc in submission.exposure_basis.keys():
                rule = class_guidelines.get(cc)
                if rule is None:
                    return False, f"Referral: Unknown class code {cc}"

                if int(rule.get("managerial", 0)) == 1:
                    thr = rule.get("incidental", 0.0)  # fraction 0..1; if missing -> 0.0
                    pct = submission.class_manual_percent(cc)  # fraction 0..1
                    if pct > thr:
                        violating.append(f"{cc} ({pct:.1%} > {thr:.0%})")
            if violating:
                return False, "Referral: Class Codes " + ", ".join(violating)
        return True, "Account is within underwriting authority"

# ------------------------------------------------------------------------------------------
class Submission():
    def __init__(self, name, exposure_basis, xmod):
        self.name = name
        self.exposure_basis = exposure_basis  #Dictionary of class codes & exposure, class code is the key with the exposure as the value. {class_code: {territory: territory, exposure: exposure}} 
        self.xmod = xmod
        self.modified_premium = self.calc_modified_premium()
        self.manual_premium = self.calc_manual_premium()

    def __repr__(self):
        exposures_str = ", ".join(
        [
            f"{cc} ({data['territory']}): {data['exposure']:,}"
            for cc, data in self.exposure_basis.items()
        ]
    )
        manual_prem = self.calc_manual_premium()
        modified_prem = self.calc_modified_premium()
        return (
            f"Submission(name='{self.name}', "
            f"xmod={self.xmod:.2f}, "
            f"manual_premium={manual_prem:,.2f}, "
            f"modified_premium={modified_prem:,.2f}, "
            f"exposures={{ {exposures_str} }})"
        )
        
    def calc_manual_premium(self): 
        # Sum of (exposure * class rate) across all class codes
        total = 0
        for class_code, entry in self.exposure_basis.items():
            rate = class_guidelines[class_code]['rate']
            exposure = entry['exposure']
            total += exposure * rate/100
        return total
        
    def calc_modified_premium(self):
        # Manual premium multiplied by xmod
        return self.calc_manual_premium() * self.xmod
        
    def class_manual_percent(self, class_code):
    #Return the manual premium % contribution for a given class code.
        if class_code not in self.exposure_basis.keys():
            raise ValueError(f"Class code {class_code} not in submission.")
        rate = class_guidelines.get(class_code, 0)['rate']
        exposure = self.exposure_basis[class_code]['exposure']
        class_premium = exposure * rate/100
        manual_premium = self.manual_premium
        if manual_premium == 0:
            return 0.0
        return class_premium / manual_premium

    def calc_territory_distribution(self):
        territory_premiums = {}
        for class_code, data in self.exposure_basis.items():
            territory = data["territory"]
            exposure = data["exposure"]

        # Get the class-specific rate from the guidelines dictionary
            class_info = class_guidelines.get(class_code)
            if not class_info:
                raise KeyError(f"Missing class code {class_code} in class_guidelines")
            rate = float(class_info["rate"])
        # Add to territory premium
            premium = (exposure * rate)/100
            territory_premiums.update({territory: 0})
            territory_premiums[territory] += premium
        total_premium = sum(territory_premiums.values())
        if total_premium <= 0.0:
        # no premium → return zero shares
            return {t: 0.0 for t in territory_premiums.keys()}
    # Convert to percentages (fractions)
        return {t: round(premium / total_premium,3) for t, premium in territory_premiums.items()}

To test these, we can define an underwriter and an underwriting manager.

The underwriter will have a premium authority up to 200,000, Xmods between 60% & 140%, authority in California and up to 25% in AOS, and a managerial level of 0 i.e. not a manager.

The manager will have a premium authority up to 1,000,000, Xmods between 40% and 200%, a, authority up to 75% in AOS, and a managerial level of 1.

In [4]:
uw = Underwriter('Underwriter', premium_authority = 200000, xmod_min_authority = 0.6, xmod_max_authority = 1.4,territory_authority = 0.25, managerial_level = 0)
mg = Underwriter('Manager', premium_authority = 1000000, xmod_min_authority = 0.4, xmod_max_authority = 2, territory_authority = 0.75, managerial_level = 1)

We will generate random businesses with arbitrary parameters and verify that our underwriters are able to make a decision on them.

For simplicity, this initial implementation will focus on California and All Other States (AOS). The selected pure premium rates are based on WCIRB workers’ compensation class codes, though this framework can be extended to include NCCI and other independent rating bureaus if needed.

The `generate_random_submission()` function will randomly select an exposure basis composed of one to four class codes, assign a random experience modification factor (Xmod), and choose territories randomly between “CA” and “AOS.”

In [5]:
def generate_random_submission():
    # Randomly pick exposure basis: 1–4 class codes
    num_classes = random.randint(1, 4)
    selected_codes = random.sample(list(class_guidelines.keys()), num_classes)
    exposure_basis = {}
    for cc in selected_codes:
        info = class_guidelines[cc]
        exposure_basis[cc] = {
            "territory": info["territory"],
            "exposure": random.randint(50_000, 2_000_000)
        }
    xmod = round(random.uniform(0.25, 1.75), 2)
    # Random territory list (e.g., 1–3 states)
    name = f"Business_{random.randint(1000, 9999)}"
    return Submission(name=name, exposure_basis=exposure_basis, xmod=xmod)

In [13]:
random.seed(42)
print(generate_random_submission())
print(uw.is_authorized(generate_random_submission()))

Submission(name='Business_4657', xmod=0.66, manual_premium=52,327.69, modified_premium=34,536.28, exposures={ 1624 (CA): 1,605,144 })
(False, 'Referral: Premium')


Above is an example of a randomly generated business. 

In the following code, we define a list called `subs` containing a list of random submissions. We will iterate `is.authorized()` over the list and compile the outputs into a pandas dataframe to analyze the results.

In [7]:
random.seed(42)
subs = []
for _ in range(20):
    subs.append(generate_random_submission())

## Testing

In [8]:
def to_row(sub, uw):
    authorized, reason = uw.is_authorized(sub)
    return {
        "name": sub.name,
        "xmod": sub.xmod,
        "manual_premium": round(sub.manual_premium, 2),
        "modified_premium": round(sub.modified_premium, 2),
        "authorized": authorized,
        "failure_reason": reason if not authorized else "",
        "n_classes": len(sub.exposure_basis),
    }

uw_authority = [to_row(sub, uw) for sub in subs]
uw_authority_df = pd.DataFrame(uw_authority)
display(uw_authority_df)

,name,xmod,manual_premium,modified_premium,authorized,failure_reason,n_classes
0,Business_4657,0.66,52327.69,34536.28,True,,1
1,Business_2424,1.59,144069.35,229070.26,False,Referral: Premium,2
2,Business_9928,1.09,184519.27,201126.01,False,Referral: Premium,4
3,Business_7924,1.46,191309.43,279311.76,False,Referral: Premium,4
4,Business_2584,0.39,121255.98,47289.83,False,Referral: Xmod,3
5,Business_9785,1.34,58129.17,77893.09,True,,3
6,Business_5803,0.37,12151.35,4496.00,False,Referral: Xmod,1
7,Business_5741,0.59,50558.83,29829.71,False,Referral: Xmod,3
8,Business_7227,1.55,33798.77,52388.10,False,Referral: Xmod,1
9,Business_5374,0.78,59554.82,46452.76,False,Referral: Territory (AOS 93.9% > 25%),3


In the above output, we can see that the underwriter does not have the authority to make a decision on the majority of incoming risks. Out of 20 submissions, there are only 4 which a decision is able to be made on without managerial approval. The most common one appears to be due to the Xmod, which does make sense as $X_{\text{mod}} \sim \text{unif}(0.25,1.75)$ and $P(0.6 < X_{\text{mod}} < 1.4) = \frac{1.4-0.6}{1.75-0.25} = \frac{8}{15} \approx 0.53$.

Business 9830 failed because of a referral class code 90501 which was more than incidental (28% > 24%). 
Business 5374 is an AOS submission with 93.9% of the premium attributable to this territory.

In [14]:
# Only in authority
display(uw_authority_df[uw_authority_df["authorized"]])

,name,xmod,manual_premium,modified_premium,authorized,failure_reason,n_classes
0,Business_4657,0.66,52327.69,34536.28,True,,1
5,Business_9785,1.34,58129.17,77893.09,True,,3
11,Business_4598,1.21,38455.86,46531.60,True,,2
17,Business_8668,1.14,1020.38,1163.23,True,,1


Now, we'll switch it to the underwriting manager to see what they have authority on

In [15]:
mg_authority = [to_row(sub, mg) for sub in subs]
mg_authority_df = pd.DataFrame(mg_authority)
display(mg_authority_df)

,name,xmod,manual_premium,modified_premium,authorized,failure_reason,n_classes
0,Business_4657,0.66,52327.69,34536.28,True,,1
1,Business_2424,1.59,144069.35,229070.26,True,,2
2,Business_9928,1.09,184519.27,201126.01,True,,4
3,Business_7924,1.46,191309.43,279311.76,True,,4
4,Business_2584,0.39,121255.98,47289.83,False,Referral: Xmod,3
5,Business_9785,1.34,58129.17,77893.09,True,,3
6,Business_5803,0.37,12151.35,4496.00,False,Referral: Xmod,1
7,Business_5741,0.59,50558.83,29829.71,False,Referral: Territory (AOS 94.6% > 75%),3
8,Business_7227,1.55,33798.77,52388.10,False,Referral: Territory (AOS 100.0% > 75%),1
9,Business_5374,0.78,59554.82,46452.76,False,Referral: Territory (AOS 93.9% > 75%),3


Authority failures are for Xmods and Territory amounts. In practice an underwriting manager likely has full AOS authority and not 75%, but class codes might be further segmented via supervisor, director, VP, etc.

# Conclusion

This project successfully developed a logical and scalable framework for automating underwriting authority checks using object-oriented programming principles. The implementation included both data and class structures that mirror real-world underwriting logic, ensuring each component remains modular, interpretable, and computationally efficient.

1. Class Design and Architecture

We defined a robust schema for Underwriter, Submission, and Guideline objects.
- Underwriter encapsulates decision authority (premium, XMOD, territory, managerial level).
- Submission models real account characteristics (class codes, exposures, territories, XMOD, and calculated premiums).
- Global dictionaries store guideline parameters (managerial flags and incidental thresholds), ensuring clarity, flexibility, and constant-time lookups.

2. Simulation and Testing
- Generated 20 randomized submissions to test the authority logic across various exposure mixes, XMODs, and territories.
- Each case was evaluated using underwriter.is_authorized(submission), confirming that the system properly identified whether an account was within authority and, when not, returned the correct referral reason.
- Results were visualized in a pandas DataFrame to ensure no missed cases or logic inconsistencies.

The resulting system demonstrates that underwriting guidelines can be efficiently modeled through structured, object-oriented logic and dictionary-based data management. The approach balances algorithmic efficiency, transparency, and real-world adaptability—providing a foundation that can scale to more complex multi-line authority frameworks or integrate seamlessly with a production underwriting platform.